In [1]:
import os
import luigi
import datetime
from google.cloud import bigquery, storage
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import re
import time
import uuid

private_key = 'serviceaccount.json'
# output_path = folder
bq_client = bigquery.Client()#.from_service_account_json(private_key)
storage_client = storage.Client()
project_id = 'projectidtest'

/Users/thesor/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/thesor/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/doc

In [2]:
filename = 'crosstabs_20190301'

In [3]:
df = pd.read_excel(filename + '.xlsx', sheet_name='Sheet1')
df.columns = ['question', 'question_ans', 'country', 'month', 'percent']
df[['question', 'question_ans', 'country']] = df[['question', 'question_ans', 'country']].fillna(method='ffill')
df = df.iloc[2:,]

df['month'] = pd.to_datetime(df['month'], format='%b - %Y')

column_order = ['country', 'month', 'question', 'question_ans', 'percent']
df = df[column_order]

df = df.set_index(['country', 'month', 'question_ans', 'question'])
df = df.sort_index()

"""
Selecting columns:
"""
df = df.unstack(['question'])
df.rename(columns={
    "Consideration/Preference": "consideration", 
    "Net Promoter Score": "nps", 
    "Aided Brand Awareness": "brand_awareness",
    "Usage - total": "usage",
}, inplace=True)
df.columns = df.columns.droplevel()
df['consideration_1_choice'] = df.loc[(slice(None), slice(None), '1st choice'), 'consideration']
df['consideration_2_choice'] = df.loc[(slice(None), slice(None), '2nd choice'), 'consideration']
df['consideration_total'] = df.loc[(slice(None), slice(None), 'Total consideration'), 'consideration']
df['nps_promoters'] = df.loc[(slice(None), slice(None), 'Promoters'), 'nps']
df['nps_passive'] = df.loc[(slice(None), slice(None), 'Passive'), 'nps']
df['nps_detractors'] = df.loc[(slice(None), slice(None), 'Detractors'), 'nps']
df['usage_most_often'] = df.loc[(slice(None), slice(None), 'Most often'), 'usage']
df['usage_usage'] = df.loc[(slice(None), slice(None), 'Usage'), 'usage']
df['usage_base'] = df.loc[(slice(None), slice(None), 'Base'), 'usage']

"""
To flatten ALL question answers into their own columns:
"""
# df = df.unstack(['question','question_ans'])
# df.rename(columns={
#     "Consideration/Preference": "consideration", 
#     "Net Promoter Score": "nps", 
#     "Total brand awareness": "brand_awareness",
#     "Usage": "usage",
# }, inplace=True)
# df.columns = df.columns.droplevel()

# regex = re.compile(r"[^\w\d]")
# columns_to_rename = []
# for i, name in enumerate(df.columns.get_level_values(1).tolist()):
#     name = name.lower()
#     name = re.sub(regex, "_", name)
#     name = df.columns.get_level_values(0).tolist()[i] + "_" + name
#     columns_to_rename.append(name)
# df.columns.set_levels(columns_to_rename, level=1, inplace=True)
# df.columns = df.columns.droplevel()

df = df / 100
df = df.fillna(value='')

df = df.reset_index()
del df.columns.name

df.to_csv(path_or_buf=filename + '.csv', index=False)
num_lines = sum(1 for line in open(filename + '.csv'))
print('Written file to ' + filename + '.csv @ Resulted CSV has ' + str(num_lines) + ' lines.')

Written file to crosstabs_20190301.csv @ Resulted CSV has 84 lines.


In [4]:
df.tail(35)

,country,month,question_ans,brand_awareness,consideration,nps,usage,consideration_1_choice,consideration_2_choice,consideration_total,nps_promoters,nps_passive,nps_detractors,usage_most_often,usage_usage,usage_base
48,Norway,2019-03-01,1st or 2nd choice,,0.0666667,,,,,,,,,,,
49,Norway,2019-03-01,2nd choice,,0.0333333,,,,0.0333333,,,,,,,
50,Norway,2019-03-01,Base,,,,0.813333,,,,,,,,,0.813333
51,Norway,2019-03-01,Detractors,,,0.25,,,,,,,0.25,,,
52,Norway,2019-03-01,Most often,,,,0.0266667,,,,,,,0.0266667,,
53,Norway,2019-03-01,Mr Green,0.613333,,,,,,,,,,,,
54,Norway,2019-03-01,Not consider,,0.833333,,,,,,,,,,,
55,Norway,2019-03-01,Passive,,,0.375,,,,,,0.375,,,,
56,Norway,2019-03-01,Promoters,,,0.375,,,,,0.375,,,,,
57,Norway,2019-03-01,Total consideration,,0.166667,,,,,0.166667,,,,,,


In [ ]:
project_id = 'project_id'
dataset_id = 'gfk'
destination_table_id = filename
bucket_name = 'bi_poc'
source_file_name = './' + filename + '.csv'
destination_blob_name = dataset_id + '/' + destination_table_id + '.csv'
# table_schema = [
#     bigquery.SchemaField('country', 'STRING'),
#     bigquery.SchemaField('month', 'STRING'),
#     bigquery.SchemaField('question_answer', 'STRING'),
#     bigquery.SchemaField('consideration', 'FLOAT'),
#     bigquery.SchemaField('nps', 'FLOAT'),
#     bigquery.SchemaField('brand_awareness', 'FLOAT'),
#     bigquery.SchemaField('usage', 'FLOAT'),
#     bigquery.SchemaField('consideration_1_choice', 'FLOAT'),
#     bigquery.SchemaField('consideration_2_choice', 'FLOAT'),
#     bigquery.SchemaField('consideration_not_consider', 'FLOAT'),
#     bigquery.SchemaField('nps_promoters', 'FLOAT'),
#     bigquery.SchemaField('nps_passive', 'FLOAT'),
#     bigquery.SchemaField('nps_detractors', 'FLOAT'),
#     bigquery.SchemaField('usage_never', 'FLOAT'),
#     bigquery.SchemaField('usage_most_often', 'FLOAT'),
#     bigquery.SchemaField('usage_past_month', 'FLOAT'),
#     bigquery.SchemaField('usage_over_month', 'FLOAT'),
# ]
table_schema = [
    bigquery.SchemaField('country', 'STRING'),
    bigquery.SchemaField('month', 'STRING'),
    bigquery.SchemaField('question_answer', 'STRING'),
    bigquery.SchemaField('brand_awareness', 'FLOAT'),
    bigquery.SchemaField('consideration', 'FLOAT'),
    bigquery.SchemaField('nps', 'FLOAT'),
    bigquery.SchemaField('usage', 'FLOAT'),
    bigquery.SchemaField('consideration_1_choice', 'FLOAT'),
    bigquery.SchemaField('consideration_2_choice', 'FLOAT'),
    bigquery.SchemaField('consideration_total', 'FLOAT'),
    bigquery.SchemaField('nps_promoters', 'FLOAT'),
    bigquery.SchemaField('nps_passive', 'FLOAT'),
    bigquery.SchemaField('nps_detractors', 'FLOAT'),
    bigquery.SchemaField('usage_most_often', 'FLOAT'),
    bigquery.SchemaField('usage_usage', 'FLOAT'),
    bigquery.SchemaField('usage_base', 'FLOAT'),
]


"""
Uploads a file to the bucket.
https://cloud.google.com/storage/docs/uploading-objects
"""
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

print('File {} uploaded to {}'.format(source_file_name, destination_blob_name))


"""
Load CSV from GCS to BQ
https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv
"""
dataset_ref = bq_client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.schema = table_schema
job_config.skip_leading_rows = 1
job_config.max_bad_records = 100
job_config.field_delimiter = ","
job_config.source_format = bigquery.SourceFormat.CSV # The source format defaults to CSV, so this line is optional.
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE # https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv#loading_csv_data_with_schema_auto-detection
job_config.allow_quoted_newlines = True

uri = 'gs://' + bucket_name + '/' + destination_blob_name

load_job = bq_client.load_table_from_uri(
    uri,
    dataset_ref.table(destination_table_id),
    job_config=job_config)  # API request
print('Starting job {} from {}'.format(load_job.job_id, uri))

load_job.result()  # Waits for table load to complete.
print('Job finished.')

destination_table = bq_client.get_table(dataset_ref.table(destination_table_id))
print('Resulted table has {} rows.'.format(destination_table.num_rows))